# tf.data を使ったテキストの読み込み

https://www.tensorflow.org/tutorials/load_data/text

In [1]:
import tensorflow as tf

import tensorflow_datasets as tfds
import os

In [3]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)

parent_dir = os.path.dirname(text_dir)

parent_dir

811008/807992 [==============================] - 0s 0us/step


'/root/.keras/datasets'

In [4]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

In [9]:
next(iter(labeled_data_sets[0].batch(5)))

(<tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b"\xef\xbb\xbfAchilles sing, O Goddess! Peleus' son;",
        b'His wrath pernicious, who ten thousand woes',
        b"Caused to Achaia's host, sent many a soul",
        b'Illustrious into Ades premature,',
        b'And Heroes gave (so stood the will of Jove)'], dtype=object)>,
 <tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 0, 0, 0])>)

In [10]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [13]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)

all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

In [14]:
for ex in all_labeled_data.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'itself.'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'quick to place himself among the foremost. The spear of King Agamemnon'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'With these may none beneath the sun compare.'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"And as a hound that, fleet of foot, o'ertakes">, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'One famed, and one a boxer never foiled;'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [16]:
tokenizer = tfds.deprecated.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

17178

In [18]:
encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set)

In [19]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

b'itself.'


In [20]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[7880]


In [21]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

In [22]:
def encode_map_fn(text, label):
  # py_func は返り値の Tensor に shape を設定しません
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` はすべての要素に shape が設定されているときにうまく動きます
  #  なので、shape を手動で設定しましょう
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

In [23]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([None],[]))

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([None],[]))

In [24]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

In [25]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

(<tf.Tensor: shape=(17,), dtype=int64, numpy=
 array([7880,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0])>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>)

In [26]:
vocab_size += 1

In [27]:
model = tf.keras.Sequential()

In [28]:
model.add(tf.keras.layers.Embedding(vocab_size, 64))

In [29]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

In [30]:
# 1 つ以上の Dense 層
# `for` 行の中のリストを編集して、層のサイズの実験をしてください
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# 出力層 最初の引数はラベルの数
model.add(tf.keras.layers.Dense(3))

In [31]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [32]:
model.fit(train_data, epochs=3, validation_data=test_data)

Epoch 1/3
697/697 [==============================] - 35s 23ms/step - loss: 0.5072 - accuracy: 0.7548 - val_loss: 0.4147 - val_accuracy: 0.8130
Epoch 2/3
697/697 [==============================] - 43s 33ms/step - loss: 0.2889 - accuracy: 0.8717 - val_loss: 0.3814 - val_accuracy: 0.8316
Epoch 3/3
697/697 [==============================] - 42s 32ms/step - loss: 0.2201 - accuracy: 0.9060 - val_loss: 0.4090 - val_accuracy: 0.8306


In [33]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

79/79 [==============================] - 4s 28ms/step - loss: 0.4090 - accuracy: 0.8306

Eval loss: 0.409, Eval accuracy: 0.831
